In [2]:
os.chdir('..')
os.listdir()

[
    '.DS_Store',
    'app',
    'LICENSE',
    'requirements.txt',
    'images',
    'Dockerfile',
    'pyproject.toml',
    'README.md',
    '.gitignore',
    'docker-compose.yml',
    '.ipynb_checkpoints',
    'poetry.lock',
    '.git',
    '.vscode',
    'data',
    'heroku.yml',
    'notebooks',
    'reports'
]

In [1]:
import csv
import os
import requests
import pandas as pd  # only through miniforge conda env
from rich import pretty
from rich.progress import track
from geopy.geocoders import ArcGIS
from geopy import distance
import numpy as np

pretty.install()

In [2]:
df = pd.read_csv('../data/CookCountyMEArchives.csv')
print(df.columns)
df.head()

Index(['Age', 'Casenumber', 'Cold Related', 'Commissioner District',
       'Death Date', 'Gender', 'Gunrelated', 'Heat Related', 'Incident City',
       'Incident Date', 'Incident Street', 'Incident Zip', 'Latino',
       'Latitude', 'Longitude', 'Manner', 'Opioids', 'Primarycause',
       'Primarycause Linea', 'Race', 'Residence City', 'Residence Zip',
       'Secondarycause'],
      dtype='object')
/Users/nick/miniforge3/envs/geo/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Age,Casenumber,Cold Related,Commissioner District,Death Date,Gender,Gunrelated,Heat Related,Incident City,Incident Date,...,Latitude,Longitude,Manner,Opioids,Primarycause,Primarycause Linea,Race,Residence City,Residence Zip,Secondarycause
0,38.0,ME2021-03896,False,5.0,2021-04-13T03:02:00.000,Male,NaN,False,PARK FOREST,2021-04-01T15:00:00.000,...,41.502759,-87.687533,NaN,NaN,NaN,NaN,Black,Park Forest,60466,NaN
1,67.0,ME2021-03895,False,NaN,2021-04-13T02:59:00.000,Female,NaN,False,DES PLAINES,2021-04-13T02:40:00.000,...,NaN,NaN,NaN,NaN,NaN,NaN,Asian,Des Plaines,60016,NaN
2,70.0,ME2021-03894,False,16.0,2021-04-13T02:02:00.000,Male,NaN,False,BERWYN,2021-02-18T15:00:00.000,...,41.843009,-87.796797,NaN,NaN,NaN,NaN,White,Berwyn,60402,NaN
3,35.0,ME2021-03893,False,6.0,2021-04-13T00:30:00.000,Female,NaN,False,LYNWOOD,2021-04-13T00:16:00.000,...,41.534453,-87.557231,NaN,NaN,NaN,NaN,Black,Lynwood,60411,NaN
4,78.0,ME2021-03892,False,NaN,2021-04-12T23:10:00.000,Male,NaN,False,HARVEY,2021-04-06T23:57:00.000,...,NaN,NaN,NaN,NaN,NaN,NaN,Black,Harvey,60426,NaN


In [3]:
df.shape

(52710, 23)

In [4]:
len(df.Casenumber.unique())

52710

In [9]:
null_street = 0
null_city = 0
null_zip = 0
null_address = 0
null_address_indices = set()
for i, row in track(df.iterrows(), total=len(df)):
    if pd.notna(row.Latitude):
        if pd.isna(row['Incident Street']):
            null_street += 1
            null_address_indices.add(i)
        if pd.isna(row['Incident City']):
            null_city += 1
            null_address_indices.add(i)
        if pd.isna(row['Incident Zip']):
            null_zip += 1
            null_address_indices.add(i)

Output()

In [10]:
print(null_city)
print(null_street)
print(null_zip)
print(null_address)

565
0
327
0


In [11]:
len(dff.Casenumber.unique())

586

In [13]:
dff = df.loc[list(null_address_indices)]
print(dff.shape)
dff.to_csv('../data/missing_addresses_with_geoms2.csv', index=False)


(586, 23)


In [4]:
newdf = df.dropna(subset=('Incident Street', 'Incident City', 'Incident Zip', 'Opioids'))
print(f"main df: {df.shape}")
print(f"new df: {newdf.shape}")
newdf.sample()

main df: (52710, 23)
new df: (50031, 23)


,Age,Casenumber,Cold Related,Commissioner District,Death Date,Gender,Gunrelated,Heat Related,Incident City,Incident Date,...,Latitude,Longitude,Manner,Opioids,Primarycause,Primarycause Linea,Race,Residence City,Residence Zip,Secondarycause
4549,62.0,ME2018-02972,False,2.0,2018-06-25T15:38:00.000,Female,False,False,CHICAGO,2018-06-25T11:38:00.000,...,41.775489,-87.641855,NATURAL,False,RUPTURED AORTIC DISSECTION,NaN,Black,Chicago,60629,NaN


In [7]:
# case number IS unique
len(newdf['Casenumber'].unique()) == newdf.shape[0]

True

In [34]:
def geo_query(row):
    street = row['Incident Street']
    city = row['Incident City']
    zip_code = row['Incident Zip']
    address = f"{street} {city} {zip_code}"
    print(address)
    url = f"https://my-geocoder.herokuapp.com/geocode?address={requests.utils.quote(address)}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise ValueError("Invalid response")


def self_geocode(address: str) -> tuple[float, float]:

    address = f"{street} {city} {zip_code}"
    coded = geocoder.geocode(address)
    return {
        "latitude": coded.latitude,
        "longitude": coded.longitude
    }

def make_address(row) -> str:
    street = row['cleaned_address']
    city = row['Incident City']
    zip_code = row['Incident Zip']
    return f"{street} {city} {zip_code}"



In [37]:
newdf['address'] = newdf.apply(lambda row: make_address(row), axis=1)
newdf.address.values[:5]

<ipython-input-37-ecc875d0720e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['address'] = newdf.apply(lambda row: make_address(row), axis=1)


array(['15300 West Avenue ORLAND PARK 60462',
       '6543 S KAMINSKI CHICAGO 60629', '15234 Paulina HARVEY 60426',
       '832 N. MOZART ST. CHICAGO 60622',
       '939 JACKSON CIRCLE ELK GROVE VILLAGE 60007'], dtype=object)

In [45]:
import re

def clean_address(x: str) -> str:
    if '(' in x:
        return re.sub(r'\(.*\)', '', x)
    else:
        return x

In [46]:
newdf['address'] = newdf.address.apply(lambda x: clean_address(x))

<ipython-input-46-e448960f1700>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['address'] = newdf.address.apply(lambda x: clean_address(x))


In [47]:
newdf[['Casenumber', 'address']].to_csv('data/new_addresses.csv', index=False)

In [48]:
newdf.shape

(50031, 24)

In [49]:
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
tqdm.pandas()

geolocator = ArcGIS()
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0)
newdf['geo_location'] = newdf['address'].progress_apply(geocode)


  0%|          | 27/52710 [00:16<8:48:52,  1.66it/s] 


KeyboardInterrupt: 

In [12]:
# attempts to make faster
import multiprocessing
from tqdm.asyncio import trange, tqdm

a_pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
result = a_pool.map(geolocator.geocode, tqdm(df['address'].values))

100%|██████████| 52710/52710 [00:00<00:00, 57406.73it/s]


KeyboardInterrupt: 

In [ ]:
result

In [50]:
import asyncio
from tqdm.asyncio import trange, tqdm
geolocator = ArcGIS()
async def code(x):
    await geolocator.geocode(x)
    print('coded')

loop = asyncio.get_event_loop()
loop.run_until_complete(code(df.address[0]))


# async for a in tqdm(df['address'].values):
#     code(a)

RuntimeError: This event loop is already running

In [ ]:
df['new_lat'] = df['geo_location'].apply(lambda x: x.latitude if x else None)
df['new_long'] = df['geo_location'].apply(lambda x: x.longitude if x else None)


In [ ]:
def calc_distance(row):
    if not pd.isna(row.new_lat) and not pd.isna(row.new_long) and not pd.isna(row.Latitude) and not pd.isna(row.Longitude):
        p1 = (row.new_lat, row.new_long)
        p2 = (row.Latitude, row.Longitude)
        d = distance.distance(p1, p2)
        return d.miles
    else:
        return None

def calc_second_distance(row):
    if not pd.isna(row.db_lat) and not pd.isna(row.db_long) and not pd.isna(row.Latitude) and not pd.isna(row.Longitude):
        p1 = (row.db_lat, row.db_long)
        p2 = (row.Latitude, row.Longitude)
        d = distance.distance(p1, p2)
        return d.miles
    else:
        return None


#df['distance'] = df.apply(lambda row: calc_distance(row), axis=1)
df['db_distance'] = df.apply(lambda row: calc_second_distance(row), axis=1)

In [ ]:
df.db_distance.values[:5]


In [ ]:
df.to_csv('data/db_coded_records.csv', index=False)

In [ ]:
import pickle

with open('data/db_dataframe.pkl', 'wb') as f:
    pickle.dump(df, f)